In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import importlib
import json
import pandas as pd
import re
import unicodedata
import traceback

from pathlib import Path
from tqdm import tqdm_notebook as tqdm

pd.set_option("display.max_colwidth", 300)

In [3]:
data_dir = Path('/data/sentiment140/')

In [4]:
train_data_path = data_dir / "training.1600000.processed.noemoticon.csv"

In [5]:
df = pd.read_csv(train_data_path.__str__(), header=None, usecols=[0, 5], encoding='latin1')
df.columns = ['polarity', 'text']
df[df["polarity"] == 0].head()
df[df["polarity"] == 4].head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


,polarity,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart."
800003,4,"Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup"
800004,4,@LovesBrooklyn2 he has that effect on everyone


In [6]:
df.polarity.value_counts()

4    800000
0    800000
Name: polarity, dtype: int64

In [7]:
CRLF_RE = re.compile(r'[\s\u3000]+')
RETWEET_RE = re.compile(r'^[rR][tT]')
HASHING_RE = re.compile(r'#[^\s]+')
MENTION_RE = re.compile(r'@[a-zA-Z0-9_]+:?')
URL_RE = re.compile(r'(?:url\s*)?(?:https?://|\w*\.\w+\.\w+)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\)…,]|[\u4E00-\u9FD0]|[あ-ん]|[\u30A1-\u30F4]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
TWEETLINK_RE = re.compile(r't.co/[a-zA-Z0-9]+')
PIC_TWITTER_RE = re.compile(r'pic.twitter.com/.+')

def clean(text, lang=None, retweet=False):
    try:
        if retweet:
            text = RETWEET_RE.sub('', text)
        elif RETWEET_RE.match(text):
            return None

        text = unicodedata.normalize('NFKC', text)
        text = CRLF_RE.sub(' ', text)
        # text = HASHING_RE.sub('', text)
        text = text.replace('#', '')
        text = MENTION_RE.sub('', text)
        text = URL_RE.sub('', text)
        text = TWEETLINK_RE.sub('', text)
        text = PIC_TWITTER_RE.sub('', text)
        text = text.strip()
        if text:
            return text
        else:
            return None
    except Exception:
        return None

In [8]:
df["clean_text"] = df.apply(lambda row: clean(row.text), axis=1)

In [9]:
df.dropna(subset=["clean_text"], inplace=True)
df.drop_duplicates(subset=["clean_text"], inplace=True)

In [10]:
n_chars = sum(len(text) for text in df.clean_text.values)
n_chars

103821369

In [11]:
n_percentage_in_monthly_budget = 200000/110/(n_chars*20/1000000)

In [12]:
bugget_for_half_of_budget = (n_chars/2)*20/1000000*110
bugget_for_half_of_budget

114203.5059

In [13]:
limit = (n_chars/2)
limit

51910684.5

In [14]:
df.polarity.value_counts()

0    779677
4    769731
Name: polarity, dtype: int64

In [15]:
neg_df = df[df["polarity"] == 0][:400000]
pos_df = df[df["polarity"] == 4][:400000]
neg_df.head()
pos_df.head()

,polarity,text,clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D","- Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.","no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


,polarity,text,clean_text
800000,4,I LOVE @Health4UandPets u guys r the best!!,I LOVE u guys r the best!!
800001,4,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!,im meeting up with one of my besties tonight! Cant wait!! - GIRL TALK!!
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart.","Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart."
800003,4,"Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup","Being sick can be really cheap when it hurts too much to eat real food Plus, your friends make you soup"
800004,4,@LovesBrooklyn2 he has that effect on everyone,he has that effect on everyone


In [16]:
neg_budget = sum(len(text) for text in neg_df.clean_text.values)*20/1000000*110
pos_budget = sum(len(text) for text in pos_df.clean_text.values)*20/1000000*110
budget = neg_budget + pos_budget
budget

117721.2432

In [17]:
import gctranslate as gct
importlib.reload(gct)

translator = gct.GCTranslate(cache_path='./gctcache.sqlite3')

<module 'gctranslate' from '/root/ELSA/dataset/sentiment140/gctranslate.py'>

In [18]:
src = "en"
tgt = "ar"

translated_neg_text = []
translated_pos_text = []

total_chars = 0

In [19]:
i = 0
pbar = tqdm(neg_df.clean_text.values)
for text in pbar:
    try:
        translated = translator.translate(text, tgt, src)
        translated_neg_text.append(translated)
    except:
        translated_neg_text.append(None)
        traceback.print_exc()
    total_chars += len(text)
    if i % 100 == 0:
        pbar.set_postfix(total_chars=str(total_chars))
    i += 1

In [ ]:
len(translated_pos_text)

In [ ]:
i = 0
pbar = tqdm(pos_df.clean_text.values)
for text in pbar:
    try:
        translated = translator.translate(text, tgt, src)
        translated_pos_text.append(translated)
    except:
        translated_pos_text.append(None)
        traceback.print_exc()
    total_chars += len(text)
    if i % 100 == 0:
        pbar.set_postfix(total_chars=str(total_chars))
    i += 1

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 379, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-20-49dbc3cda2c0>", line 5, in <module>
    translated = translator.translate(text, tgt, src)
  File "/root/ELSA/dataset/sentiment140/gctranslate.py", line 56, in translate
    translatedText = self.google_cloud_translate(inp, tgt, src, model)
  File "/root/ELSA/dataset/sentiment140/gctranslate.py", line 80, in google_cloud_translate
    inp, target_language=tgt, source_language=src, model=model)
  File "/usr/local/lib/python3.6/dist-packages/google/cloud/translate_v2/client.py", line 252, in translate
    response = self._connection.api_request(method="POST", path="", data=data)
  File "/usr/local/li